In [1]:
!pip install mysql.connector

     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     --------------------------------------- 0.0/11.9 MB 393.8 kB/s eta 0:00:30
      --------------------------------------- 0.3/11.9 MB 2.0 MB/s eta 0:00:06
     -- ------------------------------------- 0.7/11.9 MB 4.2 MB/s eta 0:00:03
     ---- ----------------------------------- 1.2/11.9 MB 5.7 MB/s eta 0:00:02
     ------ --------------------------------- 1.8/11.9 MB 6.7 MB/s eta 0:00:02
     -------- ------------------------------- 2.5/11.9 MB 7.8 MB/s eta 0:00:02
     ---------- ----------------------------- 3.0/11.9 MB 8.4 MB/s eta 0:00:02
     ----------- ---------------------------- 3.5/11.9 MB 8.7 MB/s eta 0:00:01
     -------------- ------------------------- 4.2/11.9 MB 9.3 MB/s eta 0:00:01
     ---------------- ----------------------- 4.8/11.9 MB 9.5 MB/s eta 0


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
 !pip install mysql-connector-python

  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/a1/70/618c9795fcf31768f59060db89bba7cafe9ae180064cb5206948c8c33295/mysql_connector_python-9.0.0-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB 435.7 kB/s eta 0:00:33
   ---------------------------------------- 0.1/14.3 MB 653.6 kB/s eta 0:00:22
   - -------------------------------------- 0.5/14.3 MB 2.8 MB/s eta 0:00:05
   -- ------------------------------------- 1.0/14.3 MB 5.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/14.3 MB 6.2 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/14.3 MB 7.2 MB/s eta 0:00:02
   ------- -------------------------------- 2.8/14.3 MB 8.3 MB/s eta 0:00:02
   --------- ------------------------------ 3.4/14.3 MB 9.0 MB/s eta 0:00:02
  


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('order_items.csv','order_items'),
    ('payments.csv','payments')# Added payments.csv for specific handling
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Rajan@1870',
    database='ecommerce'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/dell/Desktop/E-Commerce'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing order_items.csv
NaN values before replacement:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

Processing payments.csv
NaN values before replacement:
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

